# Small script to compare years of inventories and extrapolate the emissions


In this case we show how we use mapluft 2015 and 2020


Differences in categories between 2015 and 2020:

in 2022: 
- added ['c1307_Lieferwagen_Emissionen_Kanton', 'c1308_Reisebusse_Emissionen_Kanton']
- removed ['c3415_Kompostierung_Emissionen_Kanton']

In [ ]:
from math import floor
from pathlib import Path
import copy

from emiproc.inventories.utils import group_categories, validate_group
from emiproc.grids import SwissGrid, LV95, WGS84
from emiproc.inventories.swiss import SwissRasters
from emiproc.inventories.zurich import MapLuftZurich
from emiproc.inventories.zurich.gnrf_groups import ZH_2_GNFR
from emiproc.regrid import remap_inventory, weights_remap, get_weights_mapping
from emiproc.utilities import SEC_PER_YR
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, Point
from emiproc.inventories.utils import add_inventories, scale_inventory
from emiproc.inventories.utils import get_total_emissions
from emiproc.speciation import speciate_inventory
from emiproc.regrid import remap_inventory
from emiproc.inventories.utils import crop_with_shape
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
maplufts = {
    2015: Path(r"C:\Users\coli\Documents\MapluftAnalysis\mapLuft2015_v2020\mapLuft_2015_version_empa.gdb"),
    2020: Path(r"C:\Users\coli\Documents\MapluftAnalysis\mapLuft2020_v2021\mapLuft_2020_v2021.gdb")
}

In [ ]:
invs = {
    2015: MapLuftZurich(maplufts[2015], remove_josefstrasse_khkw = False,),
    2020: MapLuftZurich(maplufts[2020]),
}

In [ ]:
# print the missing cats for each inventories in each year
all_cats = sum([inv.categories for inv in invs.values()], [])
for year, inv in invs.items():
    print(year)
    print([cat for cat in all_cats if cat not in inv.categories ])

In [ ]:
# Deep copy the dictionary
ZH_2_GNFR_2015 = copy.deepcopy(ZH_2_GNFR)
ZH_2_GNFR_2015["GNFR_F"].remove("c1307_Lieferwagen_Emissionen_Kanton")
ZH_2_GNFR_2015["GNFR_F"].remove("c1308_Reisebusse_Emissionen_Kanton")
ZH_2_GNFR_2015["GNFR_J"].append("c3415_Kompostierung_Emissionen_Kanton")

cats_groups = {
    2015: ZH_2_GNFR_2015,
    2020: ZH_2_GNFR,
}

groupped_invs = {
    year: group_categories(inv, cats_groups[year]) for year, inv in invs.items()
}

In [ ]:
total_emissions_gnrf = {year: get_total_emissions(inv)['CO2'] for year, inv in groupped_invs.items()}
total_emissions = {year: get_total_emissions(inv)['CO2'] for year, inv in invs.items()}

In [ ]:
total_emissions_2015 = total_emissions[2015]
total_emissions_2020 = total_emissions[2020]
for cat in all_cats:
    if cat not in total_emissions_2015 or cat not in total_emissions_2020:
        continue
    # get the relative difference between 2015 and 2020
    diff = (total_emissions_2020[cat] - total_emissions_2015[cat]) / total_emissions_2020[cat]
    print(f"{cat}: {diff * 100:.2f}%")
        


In [ ]:
invs[2015].gdfs['c2301_KHKWKehricht_Emissionen_Kanton']

### Extrapolate emissions to the future

In [ ]:
total_emissions
year_future = 2022
df = pd.DataFrame(total_emissions).T / 1000
# Correct the value of 2020 because of COVID, 4% less is in the inventory
covid_correction_civil_vehicules =  1.0462
covid_correction_heavy = 1.0154
df.loc[2020, [
    'c1301_Personenwagen_Emissionen_Kanton',
    'c1303_Motorraeder_Emissionen_Kanton',
    'c1308_Reisebusse_Emissionen_Kanton', 
    'c1306_StartStopTankatmung_Emissionen_Kanton']] *= covid_correction_civil_vehicules
df.loc[2020, 'c1302_Lastwagen_Emissionen_Kanton'] *= covid_correction_heavy
# Linearly interpolate to get 2022
df.loc[year_future] = df.loc[2020] + (df.loc[2020] - df.loc[2015]) / 5 * (year_future - 2020)
# If nan assumes same as 2020
mask_nan = df.loc[year_future].isna()
df.loc[year_future, mask_nan] = df.loc[2020, mask_nan]

# Clip to 0 
df[df < 0] = 0

# Calculate the scaling factors for how to get inventory of 2022 based on 2020 

scaling_factors = df.loc[2022] / df.loc[2020]



df.plot(figsize=(10, 5), title='Total emissions in kt/year')

In [ ]:
# Scale the 2020 inventories with the calculated scaling factors to have the 2022
inv_2022 = scale_inventory(invs[2020], scaling_dict={'CO2': scaling_factors})
inv_2022_GNRF = group_categories(inv_2022, ZH_2_GNFR)

total_emissions_gnrf[2022] = get_total_emissions(inv_2022_GNRF)['CO2']

In [ ]:
# Show the differences in total emissions from the different yeas
df = pd.DataFrame(total_emissions_gnrf) / 1000
# Iterate ovr the rows of the col and format the numbers as normal floats
year = 2020
for line in df[year].iteritems():
    print(line[0], '{:.0f}'.format(line[1]), '{:.1f}%'.format(line[1] / df.loc['__total__',year] * 100), )

In [ ]:
df.plot.bar()